In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, pipeline
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from trl import SFTTrainer
from datasets import load_dataset

/media/trevor/SecondaryM2/VirginiaTech/SaTC/satc-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to('cuda:1')

# Makes training faster but a little less accurate
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name)

# setting padding instructions for tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [3]:
data_files = {'groomer_train':'../../data/groomer_train.csv', 
              'groomer_test':'../../data/groomer_test.csv', 
              'victim_train':'../../data/victim_train.csv',
              'victim_test':'../../data/victim_test.csv'}
dataset = load_dataset('csv', data_files=data_files, delimiter='|', column_names=['dialogue', 'response'])

In [4]:
def prompt_instruction_format_groomer(sample):
  task = "as an online groomer seeking to take advantage of a minor who you are talking online with"
  role = "groomer"
  
  messages = [
    {
      "role":"system",
      "content": f"You are participating in the conversation {task}. "
    },
    {
      "role": "user",
      "content": f"Generate the next {role} response in the conversation:\n{sample['dialogue']}"
    },
    {
      "role": "system",
      "content": f"{role}:{sample['response']}"
    }
  ]

  prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
  return prompt

def prompt_instruction_format_victim(sample):
  task = "as a young person talking in an online chatroom"
  role = "victim"
  
  messages = [
    {
      "role":"system",
      "content": f"You are participating in the conversation {task}. "
    },
    {
      "role": "user",
      "content": f"Generate the next {role} response in the conversation:\n{sample['dialogue']}"
    },
    {
      "role": "system",
      "content": f"{role}:{sample['response']}"
    }
  ]

  prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
  return prompt

In [5]:
role = "victim"

# Create the trainer
trainingArgs = TrainingArguments(
    output_dir=f'{role}_output',
    num_train_epochs=5,
    per_device_train_batch_size=1,
    save_strategy="epoch",
    learning_rate=2e-3
)

peft_config = LoraConfig(
      lora_alpha=16,
      lora_dropout=0.1,
      r=64,
      bias="none",
      task_type="CAUSAL_LM",
)

if role == "victim":
    form_func = prompt_instruction_format_victim
else:
    form_func = prompt_instruction_format_groomer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset[f'{role}_train'],
    eval_dataset = dataset[f'{role}_test'],
    peft_config=peft_config,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=form_func,
    args=trainingArgs,
)

/media/trevor/SecondaryM2/VirginiaTech/SaTC/satc-venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [6]:
trainer.train()

/media/trevor/SecondaryM2/VirginiaTech/SaTC/satc-venv/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
  0%|          | 0/550 [00:00<?, ?it/s]/media/trevor/SecondaryM2/VirginiaTech/SaTC/satc-venv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 20%|██        | 110/550 [11:34<33:38,  4.59s/it]Checkpoint destination directory victim_output/checkpoint-110 already exists and is non-empty. Saving will 

{'loss': 0.5407, 'grad_norm': 0.23105192184448242, 'learning_rate': 0.00018181818181818183, 'epoch': 4.55}


100%|██████████| 550/550 [57:54<00:00,  6.32s/it]

{'train_runtime': 3474.9881, 'train_samples_per_second': 0.315, 'train_steps_per_second': 0.158, 'train_loss': 0.503282803622159, 'epoch': 5.0}


TrainOutput(global_step=550, training_loss=0.503282803622159, metrics={'train_runtime': 3474.9881, 'train_samples_per_second': 0.315, 'train_steps_per_second': 0.158, 'train_loss': 0.503282803622159, 'epoch': 5.0})

In [7]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, torch_dtype=torch.float16)

In [11]:
role = "victim"

if role == "groomer":
    task = "as an online groomer seeking to take advantage of a minor who you are talking online with"
    partner = "victim"
else:
    task = "as a young person talking in an online chatroom"
    partner = "groomer"

conv = [] # "victim:how old are you;;groomer:i'm 17;;victim:are you a boy or girl"

while (1):
    u_in = input()
    if u_in == 'exit':
        break
    elif u_in == 'START':
        conv.append('START')
    else:
        print(f'{partner}:{u_in}')
        conv.append(f'{partner}:{u_in}')
    
    if len(conv) > 10:
        conversation = ';;'.join(conv[-10:])
    else:
        conversation = ';;'.join(conv)

    # print(conversation)
    messages = [
        {
          "role":"system",
          "content": f"You are participating in the conversation {task}. "
        },
        {
          "role": "user",
          "content": f"Generate the next {role} response in the conversation:\n{conversation}"
        }
      ]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=200, do_sample=True, temperature=0.8, top_k=25, top_p=0.85)
    print(f"{role}:{outputs[0]['generated_text'].split(':')[-1]}")
    conv.append(f"{outputs[0]['generated_text'].split(':')[-1]}")
    input("Press Enter")

groomer:hi
victim:hi hun
groomer:asl
victim:hiya!
groomer:hi! how old are you
victim:13
groomer:girl?
victim:nope y?
groomer:just curious. im a boy too
victim:what u mean?
groomer:i am a male
victim:a person
groomer:yes. 44 yr male
victim:o
groomer:where u live
victim:im in ca
groomer:me too. where
victim:its kinda tird i get in 2 days in kendal al thanx 2 u
groomer:what do you mean?
victim:i gotta sleep gonna get some keds 2 go 2 sleep
groomer:i see. are the keds your brothers and sisters
victim:no my bro is 2 old anyway
groomer:i see. what keds are you putting to sleep
victim:keds
groomer:who are the kids
victim:my friends
groomer:oh i see. are you having a sleepover
victim:no my friends are old
groomer:can we be friends?
victim:yah how?
groomer:i could come see you
victim:u relly want 2 meet me?
groomer:yes. i like you
victim:kewl
groomer:have you ever kissed a man?
victim:yah i got from a boyfriend 
groomer:what else did you do with your boyfriend
victim:knew a guy that done that
g

KeyboardInterrupt: Interrupted by user